## Prediction of Three Market Regimes

In [1]:
import torch.nn as nn
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
from tqdm import tqdm
import math

import logging.config
logging.config.dictConfig({'version': 1, 'disable_existing_loggers': True})

In [2]:
df = pd.read_csv('/home/lmilo_ext/Data/labeled_data_3states.csv')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df.head()

,AdrActCnt,AdrBal1in100KCnt,AdrBal1in10KCnt,AdrBal1in10MCnt,AdrBal1in1KCnt,AdrBal1in1MCnt,AdrBalNtv100Cnt,AdrBalNtv100KCnt,AdrBalNtv10Cnt,AdrBalNtv10KCnt,...,Realized Cap (7d)_btc,Realized Cap (30d)_btc,Realized Cap (90d)_btc,Realized Cap (60d)_btc,Realized Cap (180d)_btc,Realized Cap (365d)_btc,Stock to Flow ratio_btc,ReferenceRate_ma,ReferenceRate_log_return,MarketRegime
date,,,,,,,,,,,,,,,,,,,,,
2017-01-09,9.889732e+06,871727.857143,166387.428571,1.767255e+07,21509.857143,5.316332e+06,1.454555e+07,494483.857143,2.903374e+07,2.124377e+06,...,8.826018e+08,1.656658e+09,2.542547e+09,2.175202e+09,3.259974e+09,4.264022e+09,23.050918,34.808657,0.023300,1
2017-01-16,9.845421e+06,876928.142857,166228.000000,1.767422e+07,21499.857143,5.314798e+06,1.456219e+07,494053.714286,2.906592e+07,2.124969e+06,...,8.584415e+08,1.747843e+09,2.640501e+09,2.259350e+09,3.337645e+09,4.339008e+09,23.206170,34.013896,-0.023097,1
2017-01-23,9.841003e+06,882297.857143,166445.000000,1.771058e+07,21526.428571,5.318879e+06,1.456741e+07,493311.142857,2.908759e+07,2.122200e+06,...,6.752914e+08,1.736224e+09,2.654568e+09,2.270288e+09,3.359355e+09,4.354986e+09,23.586007,33.330191,-0.020306,1
2017-01-30,1.004979e+07,886092.142857,166745.857143,1.772873e+07,21664.285714,5.324168e+06,1.453396e+07,493560.142857,2.906041e+07,2.121745e+06,...,6.443858e+08,1.723002e+09,2.694323e+09,2.305593e+09,3.404450e+09,4.415296e+09,22.095225,33.232512,-0.002935,1
2017-02-06,1.017871e+07,887899.857143,167186.571429,1.774670e+07,21655.285714,5.327967e+06,1.450412e+07,494062.714286,2.904219e+07,2.125465e+06,...,6.733023e+08,1.673595e+09,2.772407e+09,2.353981e+09,3.454941e+09,4.504209e+09,20.314899,33.370602,0.004147,1


### Split for Statistical Evaluation

In [3]:
stat_eval = df.copy()
# Define the split date (06 February 2022), temporeal 80/20 split of the data
split_date = '2022-02-06'

stat_eval_test = stat_eval.loc[stat_eval.index >= split_date]
stat_eval_test.head()

,AdrActCnt,AdrBal1in100KCnt,AdrBal1in10KCnt,AdrBal1in10MCnt,AdrBal1in1KCnt,AdrBal1in1MCnt,AdrBalNtv100Cnt,AdrBalNtv100KCnt,AdrBalNtv10Cnt,AdrBalNtv10KCnt,...,Realized Cap (7d)_btc,Realized Cap (30d)_btc,Realized Cap (90d)_btc,Realized Cap (60d)_btc,Realized Cap (180d)_btc,Realized Cap (365d)_btc,Stock to Flow ratio_btc,ReferenceRate_ma,ReferenceRate_log_return,MarketRegime
date,,,,,,,,,,,,,,,,,,,,,
2022-02-07,4.984770e+07,2.658781e+06,289920.857143,5.500625e+07,48211.142857,1.465504e+07,5.832702e+07,994741.428571,1.202569e+08,6.688534e+06,...,2.437419e+10,5.583335e+10,1.275095e+11,8.726654e+10,1.844201e+11,3.411813e+11,55.668834,540.092890,-0.033330,2
2022-02-14,5.055009e+07,2.656008e+06,289738.000000,5.499573e+07,48201.428571,1.464094e+07,5.830119e+07,992627.571429,1.204062e+08,6.683030e+06,...,2.248268e+10,5.619684e+10,1.238601e+11,8.577644e+10,1.853663e+11,3.388895e+11,57.686155,528.728625,-0.021266,1
2022-02-21,5.048388e+07,2.655017e+06,289279.285714,5.496887e+07,48141.571429,1.462811e+07,5.815675e+07,987176.000000,1.203278e+08,6.649993e+06,...,2.135243e+10,5.634752e+10,1.197460e+11,8.468933e+10,1.851131e+11,3.322576e+11,54.343513,515.819066,-0.024719,1
2022-02-28,4.957744e+07,2.655316e+06,288743.000000,5.502331e+07,48196.285714,1.462820e+07,5.830249e+07,987802.714286,1.206157e+08,6.663153e+06,...,2.124967e+10,5.460837e+10,1.139341e+11,8.326354e+10,1.844852e+11,3.232177e+11,57.352742,502.690902,-0.025781,1
2022-03-07,5.083554e+07,2.655837e+06,288185.428571,5.509940e+07,48153.428571,1.463424e+07,5.849316e+07,991081.571429,1.209530e+08,6.695600e+06,...,2.647408e+10,5.576623e+10,1.107691e+11,8.471875e+10,1.857227e+11,3.150583e+11,61.099037,498.049300,-0.009276,1


### Train Test Split

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Separate features and target variable
X = df.drop("MarketRegime", axis=1)  # Features
y = df["MarketRegime"]  # Target variable

# Define the split date (06 February 2022)
split_date = '2022-02-06'

# Split the data into training and testing sets
X_train, X_test = X.loc[X.index < split_date], X.loc[X.index >= split_date]
y_train, y_test = y[X.index < split_date], y[X.index >= split_date]

### Random Forest Approach

In [5]:
import pandas as pd
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

# Define the number of splits
kf = KFold(n_splits=3)

# Initialize lists to store predictions and true labels for each fold
rf_all_y_pred = []
rf_all_y_true = []

# Perform cross-validation
for train_index, val_index in kf.split(X_train):
    X_train_cv, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_cv, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
    
    # Initialize the Random Forest classifier
    rf_classifier = RandomForestClassifier(random_state=42)
    
    # Fit the Random Forest classifier on the training data
    rf_classifier.fit(X_train_cv, y_train_cv)

    # Make predictions on the validation set
    y_pred_rf = rf_classifier.predict(X_val)

    # Append predictions and true labels for this fold
    rf_all_y_pred.extend(y_pred_rf)
    rf_all_y_true.extend(y_val)

# Calculate the overall classification report
classification_report_rf = classification_report(rf_all_y_true, rf_all_y_pred, target_names=df['MarketRegime'].unique().astype(str), digits=4)

# Print the classification report
print(classification_report_rf)

              precision    recall  f1-score   support

           1     0.8525    0.9043    0.8776       115
           0     0.7119    0.8235    0.7636       102
           2     0.9600    0.5000    0.6575        48

    accuracy                         0.8000       265
   macro avg     0.8414    0.7426    0.7663       265
weighted avg     0.8178    0.8000    0.7939       265



In [6]:
rf_final = RandomForestClassifier(random_state=42)

rf_final.fit(X_train, y_train)

# Make predictions on the test data
y_pred_test_rf = rf_final.predict(X_test)

# Calculate and print a classification report for the test data
classification_report_rf_test = classification_report(y_test, y_pred_test_rf, digits=4)

# Print the classification report for the test data
print("Classification Report for Test Data:\n", classification_report_rf_test)

Classification Report for Test Data:
               precision    recall  f1-score   support

           0     0.8889    0.9412    0.9143        17
           1     0.9583    0.6216    0.7541        37
           2     0.5200    1.0000    0.6842        13

    accuracy                         0.7761        67
   macro avg     0.7891    0.8543    0.7842        67
weighted avg     0.8557    0.7761    0.7812        67



In [7]:
stat_rf_test = stat_eval_test.copy()
# Replace the values for the actual market regime with the predicted values
stat_rf_test["MarketRegime"] = y_pred_test_rf.copy()
# Count of Bulls/Bears
count_of_zeros_rf = sum(1 for item in y_pred_test_rf if item == 0)
print("Bull:", count_of_zeros_rf)
count_of_ones_rf = sum(1 for item in y_pred_test_rf if item == 1)
print("Stagnating:", count_of_ones_rf)
count_of_twos_rf = sum(1 for item in y_pred_test_rf if item == 2)
print("Bear:", count_of_twos_rf)
# Statistics
stat_rf_test[['target']] = stat_rf_test[['ReferenceRate']].pct_change().shift(-1)
stat_rf_test.groupby('MarketRegime').agg({'target':['mean', 'std']})

Bull: 18
Stagnating: 24
Bear: 25


target          
                  mean       std
MarketRegime                    
0             0.014957  0.049051
1             0.012030  0.054134
2            -0.028740  0.072846

### Decision Tree approach

In [8]:
import pandas as pd
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier

# Define the number of splits
kf = KFold(n_splits=3)

# Initialize lists to store predictions and true labels for each fold
dt_all_y_pred = []
dt_all_y_true = []

# Perform cross-validation
for train_index, val_index in kf.split(X_train):
    X_train_cv, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_cv, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
    
    # Initialize the Random Forest classifier
    dt_classifier = DecisionTreeClassifier(random_state=42)
    
    # Fit the Random Forest classifier on the training data
    dt_classifier.fit(X_train_cv, y_train_cv)

    # Make predictions on the validation set
    y_pred_dt = dt_classifier.predict(X_val)

    # Append predictions and true labels for this fold
    dt_all_y_pred.extend(y_pred_dt)
    dt_all_y_true.extend(y_val)

# Calculate the overall classification report
classification_report_dt = classification_report(dt_all_y_true, dt_all_y_pred, target_names=df['MarketRegime'].unique().astype(str), digits=4)

# Print the classification report
print(classification_report_dt)

              precision    recall  f1-score   support

           1     0.9464    0.9217    0.9339       115
           0     0.7273    0.9412    0.8205       102
           2     0.9524    0.4167    0.5797        48

    accuracy                         0.8377       265
   macro avg     0.8754    0.7599    0.7780       265
weighted avg     0.8632    0.8377    0.8261       265



In [9]:
dt_final = DecisionTreeClassifier(random_state=42)

dt_final.fit(X_train, y_train)

# Make predictions on the test data
y_pred_test_dt = dt_final.predict(X_test)

# Calculate and print a classification report for the test data
classification_report_dt_test = classification_report(y_test, y_pred_test_dt, digits=4)

# Print the classification report for the test data
print("Classification Report for Test Data:\n", classification_report_dt_test)

Classification Report for Test Data:
               precision    recall  f1-score   support

           0     0.3721    0.9412    0.5333        17
           1     0.2778    0.1351    0.1818        37
           2     0.1667    0.0769    0.1053        13

    accuracy                         0.3284        67
   macro avg     0.2722    0.3844    0.2735        67
weighted avg     0.2801    0.3284    0.2562        67



In [10]:
stat_dt_test = stat_eval_test.copy()
# Replace the values for the actual market regime with the predicted values
stat_dt_test["MarketRegime"] = y_pred_test_dt.copy()
# Count of Bulls/Bears
count_of_zeros_dt = sum(1 for item in y_pred_test_dt if item == 0)
print("Bull:", count_of_zeros_dt)
count_of_ones_dt = sum(1 for item in y_pred_test_dt if item == 1)
print("Stagnating:", count_of_ones_dt)
count_of_twos_dt = sum(1 for item in y_pred_test_dt if item == 2)
print("Bear:", count_of_twos_dt)
# Statistics
stat_dt_test[['target']] = stat_dt_test[['ReferenceRate']].pct_change().shift(-1)
stat_dt_test.groupby('MarketRegime').agg({'target':['mean', 'std']})

Bull: 43
Stagnating: 18
Bear: 6


target          
                  mean       std
MarketRegime                    
0             0.001703  0.058313
1            -0.010547  0.076971
2            -0.011570  0.064421

### SVM approach

In [11]:
import pandas as pd
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert X_train_scaled to a Pandas DataFrame
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# Define the number of splits
kf = KFold(n_splits=3)

# Initialize lists to store predictions and true labels for each fold
svm_all_y_pred = []
svm_all_y_true = []

# Perform cross-validation
for train_index, val_index in kf.split(X_train_scaled):
    X_train_cv, X_val = X_train_scaled.iloc[train_index], X_train_scaled.iloc[val_index]
    y_train_cv, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
    
    # Initialize the SVM classifier
    svm_classifier = SVC(random_state=42, decision_function_shape='ovr')

    # Train the classifier on the training data
    svm_classifier.fit(X_train_cv, y_train_cv)
    
    # Make predictions on the validation set
    y_pred_svm = svm_classifier.predict(X_val)

    # Append predictions and true labels for this fold
    svm_all_y_pred.extend(y_pred_svm)
    svm_all_y_true.extend(y_val)

# Calculate the overall classification report
classification_report_svm = classification_report(svm_all_y_true, svm_all_y_pred, target_names=df['MarketRegime'].unique().astype(str), digits=4)

# Print the classification report
print(classification_report_svm)

              precision    recall  f1-score   support

           1     0.6724    0.6783    0.6753       115
           0     0.6275    0.6275    0.6275       102
           2     0.2979    0.2917    0.2947        48

    accuracy                         0.5887       265
   macro avg     0.5326    0.5325    0.5325       265
weighted avg     0.5873    0.5887    0.5880       265



In [12]:
svm_final = SVC(random_state=42)

svm_final.fit(X_train_scaled, y_train)

# Make predictions on the test data
y_pred_test_svm = svm_final.predict(X_test_scaled)

# Calculate and print a classification report for the test data
classification_report_svm_test = classification_report(y_test, y_pred_test_svm, digits=4)

# Print the classification report for the test data
print("Classification Report for Test Data:\n", classification_report_svm_test)

Classification Report for Test Data:
               precision    recall  f1-score   support

           0     0.3617    1.0000    0.5312        17
           1     0.0000    0.0000    0.0000        37
           2     0.4500    0.6923    0.5455        13

    accuracy                         0.3881        67
   macro avg     0.2706    0.5641    0.3589        67
weighted avg     0.1791    0.3881    0.2406        67



/home/lmilo_ext/miniconda3/envs/thesis/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lmilo_ext/miniconda3/envs/thesis/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lmilo_ext/miniconda3/envs/thesis/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

In [13]:
stat_svm_test = stat_eval_test.copy()
# Replace the values for the actual market regime with the predicted values
stat_svm_test["MarketRegime"] = y_pred_test_svm.copy()
# Count of Bulls/Bears
count_of_zeros_svm = sum(1 for item in y_pred_test_svm if item == 0)
print("Bull:", count_of_zeros_svm)
count_of_ones_svm = sum(1 for item in y_pred_test_svm if item == 1)
print("Stagnating:", count_of_ones_svm)
count_of_twos_svm = sum(1 for item in y_pred_test_svm if item == 2)
print("Bear:", count_of_twos_svm)
# Statistics
stat_svm_test[['target']] = stat_svm_test[['ReferenceRate']].pct_change().shift(-1)
stat_svm_test.groupby('MarketRegime').agg({'target':['mean', 'std']})

Bull: 47
Stagnating: 0
Bear: 20


target          
                  mean       std
MarketRegime                    
0             0.006900  0.054640
2            -0.024646  0.076946

### Logistic Regression Approach

In [14]:
import pandas as pd
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert X_train_scaled to a Pandas DataFrame
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# Define the number of splits
kf = KFold(n_splits=3)

# Initialize lists to store predictions and true labels for each fold
logreg_all_y_pred = []
logreg_all_y_true = []

# Perform cross-validation
for train_index, val_index in kf.split(X_train_scaled):
    X_train_cv, X_val = X_train_scaled.iloc[train_index], X_train_scaled.iloc[val_index]
    y_train_cv, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
    
    # Initialize the Logistic Regression classifier
    logreg_classifier = LogisticRegression(random_state=42, solver='liblinear')
    
    # Train the classifier on the training data
    logreg_classifier.fit(X_train_cv, y_train_cv)

    # Make predictions on the validation set
    y_pred_logreg = logreg_classifier.predict(X_val)

    # Append predictions and true labels for this fold
    logreg_all_y_pred.extend(y_pred_logreg)
    logreg_all_y_true.extend(y_val)

# Calculate the overall classification report
classification_report_logreg = classification_report(logreg_all_y_true, logreg_all_y_pred, target_names=df['MarketRegime'].unique().astype(str), digits=4)

# Print the classification report
print(classification_report_logreg)

              precision    recall  f1-score   support

           1     0.8605    0.6435    0.7363       115
           0     0.5238    0.3235    0.4000       102
           2     0.2500    0.6042    0.3537        48

    accuracy                         0.5132       265
   macro avg     0.5448    0.5237    0.4967       265
weighted avg     0.6203    0.5132    0.5376       265



In [15]:
logreg_final = LogisticRegression(random_state=42, solver='liblinear')

logreg_final.fit(X_train_scaled, y_train)

# Make predictions on the test data
y_pred_test_logreg = logreg_final.predict(X_test_scaled)

# Calculate and print a classification report for the test data
classification_report_logreg_test = classification_report(y_test, y_pred_test_logreg, digits=4)

# Print the classification report for the test data
print("Classification Report for Test Data:\n", classification_report_logreg_test)

Classification Report for Test Data:
               precision    recall  f1-score   support

           0     0.3111    0.8235    0.4516        17
           1     0.0000    0.0000    0.0000        37
           2     0.5455    0.9231    0.6857        13

    accuracy                         0.3881        67
   macro avg     0.2855    0.5822    0.3791        67
weighted avg     0.1848    0.3881    0.2476        67



/home/lmilo_ext/miniconda3/envs/thesis/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lmilo_ext/miniconda3/envs/thesis/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lmilo_ext/miniconda3/envs/thesis/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

In [16]:
stat_logreg_test = stat_eval_test.copy()
# Replace the values for the actual market regime with the predicted values
stat_logreg_test["MarketRegime"] = y_pred_test_logreg.copy()
# Count of Bulls/Bears
count_of_zeros_logreg = sum(1 for item in y_pred_test_logreg if item == 0)
print("Bull:", count_of_zeros_logreg)
count_of_ones_logreg = sum(1 for item in y_pred_test_logreg if item == 1)
print("Stagnating:", count_of_ones_logreg)
count_of_twos_logreg = sum(1 for item in y_pred_test_logreg if item == 2)
print("Bear:", count_of_twos_logreg)
# Statistics
stat_logreg_test[['target']] = stat_logreg_test[['ReferenceRate']].pct_change().shift(-1)
stat_logreg_test.groupby('MarketRegime').agg({'target':['mean', 'std']})

Bull: 45
Stagnating: 0
Bear: 22


target          
                  mean       std
MarketRegime                    
0             0.002446  0.052680
2            -0.012869  0.081046

### XGBoost Approach

In [17]:
import pandas as pd
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import classification_report
import xgboost as xgb

# Define the number of splits
kf = KFold(n_splits=3)

# Initialize lists to store predictions and true labels for each fold
xgb_all_y_pred = []
xgb_all_y_true = []

# Perform cross-validation
for train_index, val_index in kf.split(X_train):
    X_train_cv, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_cv, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
    
    # Initialize the XGBoost classifier
    xgb_classifier = xgb.XGBClassifier(random_state=42)
   
    # Train the classifier on the training data
    xgb_classifier.fit(X_train_cv, y_train_cv)

    # Make predictions on the validation set
    y_pred_xgb = xgb_classifier.predict(X_val)

    # Append predictions and true labels for this fold
    xgb_all_y_pred.extend(y_pred_xgb)
    xgb_all_y_true.extend(y_val)

# Calculate the overall classification report
classification_report_xgb = classification_report(xgb_all_y_true, xgb_all_y_pred, target_names=df['MarketRegime'].unique().astype(str), digits=4)

# Print the classification report
print(classification_report_xgb)

              precision    recall  f1-score   support

           1     0.9714    0.8870    0.9273       115
           0     0.8017    0.9118    0.8532       102
           2     0.8409    0.7708    0.8043        48

    accuracy                         0.8755       265
   macro avg     0.8714    0.8565    0.8616       265
weighted avg     0.8825    0.8755    0.8765       265



In [18]:
xgb_classifier_final = xgb.XGBClassifier(random_state=42)

xgb_classifier_final.fit(X_train, y_train)

# Make predictions on the test data
y_pred_test_xgb = xgb_classifier_final.predict(X_test)

# Calculate and print a classification report for the test data
classification_report_xgb_test = classification_report(y_test, y_pred_test_xgb, digits=4)

# Print the classification report for the test data
print("Classification Report for Test Data:\n", classification_report_xgb_test)

Classification Report for Test Data:
               precision    recall  f1-score   support

           0     0.4848    0.9412    0.6400        17
           1     0.9524    0.5405    0.6897        37
           2     1.0000    1.0000    1.0000        13

    accuracy                         0.7313        67
   macro avg     0.8124    0.8272    0.7766        67
weighted avg     0.8430    0.7313    0.7373        67



In [19]:
stat_xgb_test = stat_eval_test.copy()
# Replace the values for the actual market regime with the predicted values
stat_xgb_test["MarketRegime"] = y_pred_test_xgb.copy()
# Count of Bulls/Bears
count_of_zeros_xgb = sum(1 for item in y_pred_test_xgb if item == 0)
print("Bull:", count_of_zeros_xgb)
count_of_ones_xgb = sum(1 for item in y_pred_test_xgb if item == 1)
print("Stagnating:", count_of_ones_xgb)
count_of_twos_xgb = sum(1 for item in y_pred_test_xgb if item == 2)
print("Bear:", count_of_twos_xgb)
# Statistics
stat_xgb_test[['target']] = stat_xgb_test[['ReferenceRate']].pct_change().shift(-1)
stat_xgb_test.groupby('MarketRegime').agg({'target':['mean', 'std']})

Bull: 33
Stagnating: 21
Bear: 13


target          
                  mean       std
MarketRegime                    
0             0.004598  0.055942
1            -0.007358  0.061984
2            -0.013853  0.083547

### ANN Approach

In [20]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# Define the number of splits
kf = KFold(n_splits=3)

# Initialize lists to store predictions and true labels for each fold
ann_all_y_pred = []
ann_all_y_true = []

# Define ANN model
class ANNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(ANNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
input_size = X_train_scaled.shape[1]
hidden_size = 50
num_classes = 3

for train_index, val_index in kf.split(X_train):
    X_train_cv, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_cv, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Standardize the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_cv)
    X_val_scaled = scaler.transform(X_val)

    # Convert numpy arrays to PyTorch tensors
    X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train_cv.values, dtype=torch.long)
    X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val.values, dtype=torch.long)

    # Create DataLoader for training data
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    # Initialize the model
    model = ANNModel(input_size, hidden_size, num_classes)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    num_epochs = 100
    for epoch in range(num_epochs):
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    model.eval()
    with torch.no_grad():
        y_pred_probs = model(X_val_tensor)
        _, y_pred_ann = torch.max(y_pred_probs, 1)

    # Convert predictions to numpy arrays
    y_pred_ann = y_pred_ann.numpy()

    # Append predictions and true labels for this fold
    ann_all_y_pred.extend(y_pred_ann)
    ann_all_y_true.extend(y_val)

# Calculate the overall classification report
classification_report_ann = classification_report(ann_all_y_true, ann_all_y_pred, target_names=df['MarketRegime'].unique().astype(str), digits=4)

# Print the classification report
print(classification_report_ann)

              precision    recall  f1-score   support

           1     0.6882    0.5565    0.6154       115
           0     0.4599    0.6176    0.5272       102
           2     0.6286    0.4583    0.5301        48

    accuracy                         0.5623       265
   macro avg     0.5922    0.5442    0.5576       265
weighted avg     0.5895    0.5623    0.5660       265



In [21]:
# Standardize the test features
X_test_scaled = scaler.transform(X_test)

# Convert numpy arrays to PyTorch tensors
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Initialize a list to store predictions on the test data
y_pred_test_ann = []

# Set the model to evaluation mode
model.eval()
with torch.no_grad():
    y_pred_probs_test = model(X_test_tensor)
    _, y_pred_test = torch.max(y_pred_probs_test, 1)

    # Convert predictions to numpy arrays
    y_pred_test_ann = y_pred_test.numpy()

# Calculate and print a classification report for the test data
classification_report_ann_test = classification_report(y_test, y_pred_test_ann, digits=4)

# Print the classification report for the test data
print("Classification Report for Test Data:\n", classification_report_ann_test)

Classification Report for Test Data:
               precision    recall  f1-score   support

           0     0.5185    0.8235    0.6364        17
           1     0.6000    0.6486    0.6234        37
           2     0.0000    0.0000    0.0000        13

    accuracy                         0.5672        67
   macro avg     0.3728    0.4907    0.4199        67
weighted avg     0.4629    0.5672    0.5057        67



/home/lmilo_ext/miniconda3/envs/thesis/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lmilo_ext/miniconda3/envs/thesis/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lmilo_ext/miniconda3/envs/thesis/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

In [22]:
stat_ann_test = stat_eval_test.copy()
# Replace the values for the actual market regime with the predicted values
stat_ann_test["MarketRegime"] = y_pred_test_ann.copy()
# Count of Bulls/Bears
count_of_zeros_ann = sum(1 for item in y_pred_test_ann if item == 0)
print("Bull:", count_of_zeros_ann)
count_of_ones_ann = sum(1 for item in y_pred_test_ann if item == 1)
print("Stagnating:", count_of_ones_ann)
count_of_twos_ann = sum(1 for item in y_pred_test_ann if item == 2)
print("Bear:", count_of_twos_ann)
# Statistics
stat_ann_test[['target']] = stat_ann_test[['ReferenceRate']].pct_change().shift(-1)
stat_ann_test.groupby('MarketRegime').agg({'target':['mean', 'std']})

Bull: 27
Stagnating: 40
Bear: 0


target          
                  mean       std
MarketRegime                    
0             0.011875  0.054193
1            -0.012106  0.067602

### LSTM Approach

In [23]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert X_train_scaled to a Pandas DataFrame
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)


# Define LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True, num_layers=2)  # Two layers
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, :])
        return out

input_size = X_train_scaled.shape[1]
hidden_size = 50
num_classes = 3  # Assuming binary classification

# Define the number of splits
kf = KFold(n_splits=3)

# Initialize lists to store predictions and true labels for each fold
lstm_all_y_pred = []
lstm_all_y_true = []

# Perform cross-validation
for train_index, val_index in kf.split(X_train):
    X_train_cv, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_cv, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Standardize the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_cv)
    X_val_scaled = scaler.transform(X_val)

    # Convert numpy arrays to PyTorch tensors
    X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train_cv.values, dtype=torch.long)
    X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val.values, dtype=torch.long)

    # Create DataLoader for training data
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    
    # Initialize the LSTM model
    model = LSTMModel(input_size, hidden_size, num_classes)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    num_epochs = 100
    for epoch in range(num_epochs):
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluate the model
    model.eval()
    with torch.no_grad():
        y_pred_probs = model(X_val_tensor)
        _, y_pred_lstm = torch.max(y_pred_probs, 1)

    # Convert predictions to numpy arrays
    y_pred_lstm = y_pred_lstm.numpy()

    # Append predictions and true labels for this fold
    lstm_all_y_pred.extend(y_pred_lstm)
    lstm_all_y_true.extend(y_val)

# Calculate the overall classification report
classification_report_lstm = classification_report(lstm_all_y_true, lstm_all_y_pred, target_names=df['MarketRegime'].unique().astype(str), digits=4)

# Print the classification report
print(classification_report_lstm)

              precision    recall  f1-score   support

           1     0.7103    0.6609    0.6847       115
           0     0.5537    0.6569    0.6009       102
           2     0.4054    0.3125    0.3529        48

    accuracy                         0.5962       265
   macro avg     0.5565    0.5434    0.5462       265
weighted avg     0.5948    0.5962    0.5923       265



In [24]:
# Standardize the test features
X_test_scaled = scaler.transform(X_test)

# Convert numpy arrays to PyTorch tensors
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Initialize a list to store predictions on the test data
y_pred_test_lstm = []

# Set the model to evaluation mode
model.eval()
with torch.no_grad():
    y_pred_probs_test = model(X_test_tensor)
    _, y_pred_test = torch.max(y_pred_probs_test, 1)

    # Convert predictions to numpy arrays
    y_pred_test_lstm = y_pred_test.numpy()

# Calculate and print a classification report for the test data
classification_report_lstm_test = classification_report(y_test, y_pred_test_lstm, digits=4)

# Print the classification report for the test data
print("Classification Report for Test Data:\n", classification_report_lstm_test)

Classification Report for Test Data:
               precision    recall  f1-score   support

           0     0.6364    0.8235    0.7179        17
           1     0.6667    0.8108    0.7317        37
           2     0.0000    0.0000    0.0000        13

    accuracy                         0.6567        67
   macro avg     0.4343    0.5448    0.4832        67
weighted avg     0.5296    0.6567    0.5862        67



/home/lmilo_ext/miniconda3/envs/thesis/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lmilo_ext/miniconda3/envs/thesis/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lmilo_ext/miniconda3/envs/thesis/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

In [25]:
stat_lstm_test = stat_eval_test.copy()
# Replace the values for the actual market regime with the predicted values
stat_lstm_test["MarketRegime"] = y_pred_test_lstm.copy()
# Count of Bulls/Bears
count_of_zeros_lstm = sum(1 for item in y_pred_test_lstm if item == 0)
print("Bull:", count_of_zeros_lstm)
count_of_ones_lstm = sum(1 for item in y_pred_test_lstm if item == 1)
print("Stagnating:", count_of_ones_lstm)
count_of_twos_lstm = sum(1 for item in y_pred_test_lstm if item == 2)
print("Bear:", count_of_twos_lstm)
# Statistics
stat_lstm_test[['target']] = stat_lstm_test[['ReferenceRate']].pct_change().shift(-1)
stat_lstm_test.groupby('MarketRegime').agg({'target':['mean', 'std']})

Bull: 22
Stagnating: 45
Bear: 0


target          
                  mean       std
MarketRegime                    
0             0.014401  0.063046
1            -0.010621  0.062569